In [1]:
%load_ext autoreload
%autoreload 2

here the objective is to fit the logit graph model to multiple different graph domains and then visualize the original vs fitted graphs

In [2]:

import sys
import os
sys.path.append('../')

#Graph imports
import src.graph as graph
import src.logit_estimator as estimator
import src.utils as utils
import src.model_selection as model_selection
import src.gic as gic
import src.param_estimator as pe
import src.graph as graph
import src.model_selection as ms

# usual imports
import matplotlib.pyplot as plt
import pickle
import math
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import gaussian_kde
import numpy as np
import pandas as pd
import seaborn as sns
import gc
import random
import networkx as nx
from numpy import errstate

from IPython.display import display
from pyvis.network import Network
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [3]:
def load_facebook_graph(data_dir, graph_id):
    """
    Loads a Facebook graph from the specified directory and graph ID.

    Args:
        data_dir (str): The directory where the facebook data is stored.
        graph_id (int): The ID of the graph to load.

    Returns:
        networkx.Graph: The loaded graph as a networkx Graph object.
    """
    edge_file = os.path.join(data_dir, f"{graph_id}.edges")
    
    G = nx.Graph()
    
    with open(edge_file, 'r') as f:
        for line in f:
            # The files are space-separated, so we split the line
            # and convert the node IDs to integers.
            u, v = map(int, line.strip().split())
            G.add_edge(u, v)
            
    return G


data_directory = f'../data/misc/facebook'
ids = [0, 107, 348, 414, 686, 698, 1684, 1912, 3437, 3980]
graphs = []
for id in ids:
    graphs.append(load_facebook_graph(data_directory, id))
    print(f"Graph {id} loaded successfully. Edges: {graphs[-1].number_of_edges()}, Nodes: {graphs[-1].number_of_nodes()}")

Graph 0 loaded successfully. Edges: 2519, Nodes: 333
Graph 107 loaded successfully. Edges: 26749, Nodes: 1034
Graph 348 loaded successfully. Edges: 3192, Nodes: 224
Graph 414 loaded successfully. Edges: 1693, Nodes: 150
Graph 686 loaded successfully. Edges: 1656, Nodes: 168
Graph 698 loaded successfully. Edges: 270, Nodes: 61
Graph 1684 loaded successfully. Edges: 14024, Nodes: 786
Graph 1912 loaded successfully. Edges: 30025, Nodes: 747
Graph 3437 loaded successfully. Edges: 4813, Nodes: 534
Graph 3980 loaded successfully. Edges: 146, Nodes: 52


In [25]:
FOLDER_OUTPUT = f'runs/fitted_graphs_comparison_facebook2'
out_files = os.listdir(FOLDER_OUTPUT)
out_files

['summary_dfs_698.pkl',
 'fitted_graphs_3980.pkl',
 'summary_dfs_3980.pkl',
 'fitted_graphs_698.pkl']

In [31]:
import pickle
# with open(f'{FOLDER_OUTPUT}/fitted_graphs_698.pkl', 'rb') as f:
#     fitted_graphs = pickle.load(f)
# with open(f'{FOLDER_OUTPUT}/summary_dfs_698.pkl', 'rb') as f:
#     summary_dfs = pickle.load(f)
with open(f'{FOLDER_OUTPUT}/comparators_107.pkl', 'rb') as f:
    comparators = pickle.load(f)


In [32]:
len(comparators)

10

In [34]:
comparators[0].summary_df

,graph_filename,model,gic_value,param,fit_success,nodes,edges,density,avg_clustering,avg_path_length,diameter,assortativity,num_components,largest_component_size
0,3980.edges,Original,NaN,N/A,True,52,146,0.110106,0.461727,2.566596,5,0.160451,4,44
1,3980.edges,LG,0.606933,"d=1, sigma=-4.3053",True,52,152,0.114630,0.117589,2.349020,4,-0.097657,2,51
2,3980.edges,WS,0.128505,40.2,True,52,156,0.117647,0.075788,2.325792,4,-0.012178,1,52
3,3980.edges,ER,0.570055,0.115556,True,52,148,0.111614,0.117853,2.457768,5,-0.030350,1,52
4,3980.edges,BA,0.583533,3.333333,True,52,147,0.110860,0.140140,2.334842,4,-0.122385,1,52
5,3980.edges,GRG,13.273996,1.0,True,52,1304,0.983409,0.985094,1.016591,2,-0.054451,1,52
